<a href="https://colab.research.google.com/github/PriyankaSharma0925/NLP/blob/main/Introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow
tensorflow.__version__

'2.8.0'

In [1]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.8.0


In [3]:
pip install tensorflow==2.7

     |████████████████████████████████| 489.6 MB 24 kB/s 
     |████████████████████████████████| 463 kB 60.3 MB/s 
     |████████████████████████████████| 1.3 MB 68.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [4]:

# NLP problem also known as seq2seq

#Check the GPU

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-14f6ef5d-0057-ad21-c08b-49f7d91fd92f)


In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-05-10 06:41:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-10 06:41:15 (113 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

In [7]:
#importing the dataset: Kaggle's Introduction to NLP 

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


--2022-05-10 06:41:21--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.144, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-05-10 06:41:21 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [8]:
unzip_data('nlp_getting_started.zip')

Visualizing the text data.

In [9]:
import pandas as pd

In [10]:
train_df=pd.read_csv("train.csv")

In [11]:
test_df=pd.read_csv("test.csv")

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
shuffled_train_dataset=train_df.sample(frac=1,random_state=42)
shuffled_train_dataset.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


**Split the training data into training and validation set.**

In [117]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(shuffled_train_dataset['text'].to_numpy(),shuffled_train_dataset['target'].to_numpy(),test_size=0.1,random_state=42)
len(X_train)

6851

**Creating features for test dataset**

In [98]:
import tensorflow as tf

In [99]:
from tensorflow.keras.layers import TextVectorization

In [100]:
textvectorizer=TextVectorization(output_sequence_length=15,pad_to_max_tokens=True,max_tokens=1000)

In [101]:
sent=shuffled_train_dataset['text'][0]

In [102]:
textvectorizer.adapt(X_train)

In [103]:
textvectorizer("Its raining heavily outside")

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 38,   1,   1, 680,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [104]:
import random

In [105]:
rand_sent=random.choice(X_train)
rand_sent

'The thing with rules is break it once it becomes easier the next time.\nhttp://t.co/hGb1mc3IRk  https://t.co/6ysXGhc8gz'

In [106]:
textvectorizer(rand_sent)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  2, 451,  14, 989,   9,   1,  15, 769,  15,   1,   1,   2, 228,
        90,   1])>

**Get the total unique words.**

In [107]:
vocab=textvectorizer.get_vocabulary()
print(vocab)

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'amp', 'your', 'not', 'out', 'after', 'its', 'all', 'no', 'will', 'has', 'fire', 'when', 'get', 'an', 'if', 'we', 'now', 'new', 'via', 'about', 'more', 'or', 'news', 'what', 'dont', 'he', 'been', 'they', 'one', 'people', 'how', 'over', 'who', 'into', 'do', 'were', 'us', 'video', 'can', 'emergency', '2', 'there', 'disaster', 'police', 'would', 'still', 'than', 'her', 'some', 'his', 'body', 'crash', 'back', 'california', 'got', 'burning', 'why', 'time', 'off', 'suicide', 'storm', 'buildings', 'know', 'day', 'them', 'man', 'had', 'see', 'love', 'first', 'going', 'cant', 'rt', 'world', 'two', 'bomb', 'fires', 'attack', '3', 'being', 'train', 'our', 'nuclear', 'go', 'full', 'killed', 'dead', 'car', 'their', 'accident', 'good', 'may', 'many', 'youtube', 'watch', 'only', 'life'

In [108]:
type(vocab)

list

In [109]:
vocab[-5:]

['park', 'pakistani', 'outrage', 'niggas', 'muslims']

**#Adding the embedding layer**

In [110]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=len(vocab),output_dim=128,input_length=15)
embedding

In [111]:
rand_sent=random.choice(X_train)
embedding(textvectorizer(rand_sent))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.00721097,  0.00038321,  0.02200569, ...,  0.03611131,
        -0.02069273, -0.02372407],
       [ 0.00721097,  0.00038321,  0.02200569, ...,  0.03611131,
        -0.02069273, -0.02372407],
       [ 0.00721097,  0.00038321,  0.02200569, ...,  0.03611131,
        -0.02069273, -0.02372407],
       ...,
       [ 0.03171219,  0.02181066,  0.02732604, ...,  0.04336467,
        -0.04466187, -0.02205631],
       [ 0.00721097,  0.00038321,  0.02200569, ...,  0.03611131,
        -0.02069273, -0.02372407],
       [ 0.03459915, -0.03888029, -0.04999318, ...,  0.02634645,
        -0.0388489 ,  0.02856988]], dtype=float32)>

**Model 0 : Getting a baseline. For this we will be using the multinomial naive bayes using the tf-idf**

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline

In [118]:
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clfier",MultinomialNB())
])
model_0.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clfier', MultinomialNB())])

**Evaluate our baseline model.**

In [120]:
baseline_score=model_0.score(X_val,y_val)
print(f"Baseline method has achieved the accuracy of {baseline_score*100:.2f}%")

Baseline method has achieved the accuracy of 79.27%


In [123]:
# Make predictions:

model_0_predictions=model_0.predict(X_val)
model_0_predictions[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [131]:
# function to evaluate accuracy,recall,predictions and F1-score
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def scorecard(y_pred,y_true,model):
  model_accuracy=accuracy_score(y_true,y_pred)
  print(f"The accuracy score for {model} is {model_accuracy*100:.2f}%")
  precision,recall,f1_score,_=precision_recall_fscore_support(y_true,y_pred,)
  print(f"The precision score for {model} is {precision},recall is {recall} and f1_score is {f1_score}")
  model_result={
      'Accuracy':model_accuracy,
      'Precision':precision,
      'Recall':recall,
      'F1_score':f1_score
  }
  return model_result


In [132]:
scorecard(model_0_predictions,y_val,"model_0")

The accuracy score for model_0 is 79.27%
The precision score for model_0 is [0.74806202 0.88617886],recall is [0.93236715 0.62643678] and f1_score is [0.83010753 0.73400673]


{'Accuracy': 0.7926509186351706,
 'F1_score': array([0.83010753, 0.73400673]),
 'Precision': array([0.74806202, 0.88617886]),
 'Recall': array([0.93236715, 0.62643678])}